Code taken from the README file, adapted to use weighted network

In [5]:
import numpy as np
import pandas as pd

src = '../data/raw_networks' # root dir of data
meta = pd.read_csv('../data/network_list.csv')
roles = pd.read_csv('../data/node_role.csv')

def loadGame(game, is_weighted):
    # N is the number of players in the game
    # load csv data
    # unweighted and weighted networks can be loaded exactly the same way
    # below shows the loader for weighted networks
    
    if is_weighted:
        df_network = pd.read_csv(f"{src}/network{game}_weighted.csv", index_col=0)
    else:
        df_network = pd.read_csv(f"{src}/network{game}.csv", index_col=0)
    # for dense repr
    #df_network = pd.read_csv(f"{src}/network{game}_weighted.csv", index_col=0)
    
    # get meta NUMBER_OF_PARTICIPANTS by game NETWORK id
    N = meta.loc[game, "NUMBER_OF_PARTICIPANTS"]

    # T is number of timestamps (10 frames)
    T = len(df_network)
    # load VFOA network to T x N x (N+1) array
    # vfoa[t, n, i] is the probability of player n+1 looking at object i at time t
    # i: 0 - laptop, 1 - player 1, 2 - player 2, ..., N - player N
    vfoa = np.reshape(df_network.values, (T, N, N + 1))

    # print information
    print(f"network id:{game}\t length(x 1/3 second): {T}\t num of players: {N}")
    return vfoa

In [10]:
for _, meta_row in meta.iterrows():
    game_id = meta_row['NETWORK']
    if len(roles[roles['ID'] == game_id]) == 0:
        print(f"No roles for game {game_id}")
        continue
        
    #game = loadGame(game_id, 0)
    game_weighted = loadGame(game_id, 1)
    
    records = []

    #t, n1, n2 = game.shape
    t, n1, n2 = game_weighted.shape

    # this is 0-indexed while nodes are 1-indexed
    game_roles = {i: roles[
                (roles["node"] == f"P{i+1}") & (roles["ID"] == game_id)
            ]["role"].values[0] for i in range(n1)}

    records = []
    for timestep, frame in enumerate(game_weighted):
        #print(f"timestep {timestep} {frame}")

        # get max index of each row 
        max_index = np.argmax(frame, axis=1)
        for i, item_id in enumerate(max_index):
            # get the role of user i in game game_id
            role = game_roles[i]
            list_of_features = frame[i].tolist()

            # record: user_id, item_id, timestamp, state_label,  feature
            records.append([i + 1, item_id, timestep, role] + list_of_features)

    
    game_df = pd.DataFrame(records)
    game_df.columns = ["user_id", "item_id", "timestamp", "state_label"] + [f"feature {i}" for i in range(len(game_df.columns) - 4)]
    game_df.to_csv(f"../data/processed_networks/network_{game_id}.csv", index=False, header=False)

network id:0	 length(x 1/3 second): 7323	 num of players: 7
No roles for game 1
network id:2	 length(x 1/3 second): 7533	 num of players: 7
network id:3	 length(x 1/3 second): 6264	 num of players: 7
network id:4	 length(x 1/3 second): 7323	 num of players: 8
network id:5	 length(x 1/3 second): 9651	 num of players: 8
network id:6	 length(x 1/3 second): 7260	 num of players: 7
network id:7	 length(x 1/3 second): 7245	 num of players: 6
network id:8	 length(x 1/3 second): 6243	 num of players: 7
No roles for game 9
network id:10	 length(x 1/3 second): 6681	 num of players: 7
network id:11	 length(x 1/3 second): 5415	 num of players: 7
network id:12	 length(x 1/3 second): 5571	 num of players: 8
network id:13	 length(x 1/3 second): 7173	 num of players: 7
network id:14	 length(x 1/3 second): 9429	 num of players: 8
network id:15	 length(x 1/3 second): 5466	 num of players: 6
network id:16	 length(x 1/3 second): 6678	 num of players: 6
network id:17	 length(x 1/3 second): 3054	 num of pla

Flatten the adjacency matrix so that each row represents the gaze vector of the user.
Convert this into the wikipedia format and save as CSV

In [4]:
import os

# create `network_processed` directory if it doesn't exist
if not os.path.exists("network_weighted_processed"):
    os.makedirs("network_weighted_processed")

In [12]:
game = loadGame(0, 1)
for timestep, frame in enumerate(game):
        print(f"timestep {timestep} {frame}")

network id:0	 length(x 1/3 second): 7323	 num of players: 7
timestep 0 [[0.534 0.    0.046 0.083 0.    0.041 0.029 0.067]
 [0.077 0.023 0.    0.127 0.011 0.025 0.034 0.503]
 [0.079 0.03  0.041 0.    0.039 0.041 0.037 0.533]
 [0.1   0.049 0.019 0.063 0.    0.073 0.377 0.12 ]
 [0.043 0.034 0.04  0.357 0.02  0.    0.05  0.256]
 [0.15  0.111 0.131 0.128 0.044 0.115 0.    0.12 ]
 [0.094 0.071 0.136 0.172 0.053 0.165 0.109 0.   ]]
timestep 1 [[0.646 0.    0.061 0.123 0.    0.05  0.029 0.091]
 [0.086 0.021 0.    0.164 0.014 0.043 0.043 0.629]
 [0.169 0.037 0.047 0.    0.057 0.053 0.043 0.594]
 [0.014 0.056 0.013 0.039 0.    0.07  0.653 0.156]
 [0.029 0.043 0.057 0.443 0.029 0.    0.054 0.346]
 [0.116 0.08  0.414 0.151 0.034 0.085 0.    0.12 ]
 [0.017 0.05  0.45  0.178 0.051 0.182 0.071 0.   ]]
timestep 2 [[0.73  0.    0.043 0.088 0.006 0.041 0.021 0.071]
 [0.086 0.014 0.    0.173 0.014 0.041 0.043 0.629]
 [0.109 0.037 0.056 0.    0.067 0.06  0.041 0.63 ]
 [0.014 0.057 0.014 0.043 0.    0.071 

timestep 340 [[0.017 0.    0.114 0.152 0.007 0.127 0.054 0.529]
 [0.    0.    0.    0.646 0.049 0.039 0.009 0.257]
 [0.03  0.041 0.054 0.    0.461 0.157 0.11  0.146]
 [0.414 0.06  0.036 0.084 0.    0.079 0.134 0.193]
 [0.159 0.084 0.109 0.18  0.017 0.    0.027 0.424]
 [0.096 0.041 0.049 0.063 0.    0.068 0.    0.683]
 [0.021 0.044 0.114 0.144 0.054 0.21  0.411 0.   ]]
timestep 341 [[0.024 0.    0.114 0.156 0.004 0.128 0.044 0.529]
 [0.    0.    0.    0.669 0.037 0.047 0.009 0.239]
 [0.    0.027 0.034 0.    0.609 0.127 0.083 0.12 ]
 [0.514 0.053 0.033 0.104 0.    0.094 0.043 0.159]
 [0.131 0.083 0.117 0.209 0.034 0.    0.026 0.4  ]
 [0.093 0.043 0.053 0.07  0.    0.064 0.    0.677]
 [0.077 0.034 0.09  0.12  0.05  0.107 0.521 0.   ]]
timestep 342 [[0.03  0.    0.113 0.145 0.006 0.128 0.05  0.529]
 [0.036 0.01  0.    0.474 0.031 0.125 0.043 0.281]
 [0.    0.027 0.043 0.    0.609 0.131 0.083 0.107]
 [0.474 0.05  0.03  0.104 0.    0.104 0.07  0.167]
 [0.091 0.066 0.113 0.261 0.026 0.    0.0

 [0.013 0.683 0.057 0.11  0.014 0.108 0.014 0.   ]]
timestep 645 [[0.017 0.    0.111 0.16  0.01  0.115 0.063 0.524]
 [0.081 0.014 0.    0.177 0.027 0.034 0.03  0.636]
 [0.046 0.244 0.263 0.    0.063 0.207 0.051 0.126]
 [0.497 0.063 0.051 0.073 0.    0.084 0.093 0.139]
 [0.024 0.084 0.106 0.327 0.036 0.    0.02  0.403]
 [0.001 0.031 0.026 0.72  0.034 0.003 0.    0.184]
 [0.006 0.699 0.053 0.089 0.014 0.123 0.017 0.   ]]
timestep 646 [[0.029 0.    0.111 0.163 0.016 0.117 0.039 0.526]
 [0.027 0.01  0.    0.294 0.056 0.142 0.167 0.304]
 [0.011 0.033 0.043 0.    0.564 0.161 0.114 0.073]
 [0.53  0.074 0.039 0.09  0.    0.087 0.106 0.074]
 [0.037 0.079 0.093 0.341 0.046 0.    0.023 0.381]
 [0.003 0.031 0.029 0.797 0.043 0.    0.    0.097]
 [0.016 0.267 0.207 0.318 0.03  0.128 0.034 0.   ]]
timestep 647 [[0.054 0.    0.107 0.195 0.027 0.126 0.041 0.449]
 [0.004 0.    0.    0.309 0.047 0.291 0.039 0.31 ]
 [0.    0.031 0.041 0.    0.614 0.163 0.101 0.049]
 [0.199 0.064 0.023 0.069 0.    0.094 0.

timestep 936 [[0.019 0.    0.106 0.135 0.001 0.078 0.036 0.626]
 [0.096 0.017 0.    0.04  0.003 0.014 0.02  0.81 ]
 [0.054 0.084 0.051 0.    0.177 0.141 0.113 0.379]
 [0.261 0.044 0.036 0.08  0.    0.077 0.204 0.297]
 [0.03  0.089 0.1   0.043 0.016 0.    0.037 0.686]
 [0.031 0.024 0.054 0.056 0.011 0.034 0.    0.789]
 [0.03  0.014 0.071 0.057 0.029 0.057 0.741 0.   ]]
timestep 937 [[0.026 0.    0.106 0.151 0.    0.088 0.054 0.576]
 [0.127 0.009 0.    0.047 0.003 0.023 0.019 0.773]
 [0.037 0.041 0.047 0.    0.226 0.22  0.089 0.34 ]
 [0.051 0.043 0.033 0.308 0.    0.161 0.024 0.38 ]
 [0.033 0.091 0.1   0.043 0.016 0.    0.04  0.677]
 [0.046 0.08  0.08  0.101 0.029 0.056 0.    0.609]
 [0.031 0.014 0.071 0.057 0.029 0.057 0.74  0.   ]]
timestep 938 [[0.    0.    0.01  0.006 0.    0.    0.003 0.981]
 [0.1   0.    0.    0.    0.    0.    0.    0.9  ]
 [0.039 0.013 0.033 0.    0.034 0.576 0.034 0.271]
 [0.043 0.06  0.037 0.076 0.    0.091 0.026 0.667]
 [0.019 0.049 0.086 0.023 0.013 0.    0.0

timestep 1276 [[0.001 0.    0.013 0.009 0.    0.001 0.001 0.974]
 [0.033 0.673 0.    0.013 0.    0.009 0.    0.273]
 [0.    0.    0.01  0.    0.    0.    0.    0.99 ]
 [0.566 0.037 0.043 0.014 0.    0.057 0.043 0.24 ]
 [0.009 0.019 0.033 0.01  0.007 0.    0.033 0.89 ]
 [0.    0.    0.007 0.    0.    0.    0.    0.993]
 [0.014 0.683 0.056 0.1   0.    0.11  0.037 0.   ]]
timestep 1277 [[0.014 0.    0.106 0.148 0.    0.115 0.041 0.576]
 [0.1   0.31  0.    0.122 0.009 0.037 0.011 0.411]
 [0.481 0.043 0.034 0.    0.043 0.061 0.016 0.321]
 [0.567 0.044 0.046 0.03  0.    0.057 0.063 0.193]
 [0.016 0.039 0.033 0.309 0.029 0.    0.146 0.43 ]
 [0.079 0.033 0.066 0.084 0.003 0.064 0.    0.671]
 [0.014 0.567 0.08  0.155 0.004 0.139 0.04  0.   ]]
timestep 1278 [[0.014 0.    0.116 0.169 0.016 0.124 0.033 0.529]
 [0.044 0.18  0.    0.466 0.056 0.094 0.004 0.156]
 [0.46  0.037 0.039 0.    0.063 0.077 0.036 0.289]
 [0.59  0.047 0.056 0.043 0.    0.071 0.121 0.071]
 [0.014 0.029 0.04  0.551 0.034 0.    

...

timestep 6414 [[0.003 0.    0.11  0.626 0.019 0.086 0.041 0.116]
 [0.064 0.017 0.    0.376 0.039 0.298 0.121 0.084]
 [0.02  0.03  0.057 0.    0.371 0.25  0.167 0.104]
 [0.024 0.05  0.019 0.064 0.    0.06  0.299 0.484]
 [0.029 0.036 0.021 0.241 0.61  0.    0.009 0.054]
 [0.031 0.029 0.061 0.534 0.036 0.065 0.    0.244]
 [0.    0.671 0.043 0.102 0.014 0.141 0.029 0.   ]]
timestep 6415 [[0.004 0.    0.084 0.722 0.014 0.071 0.021 0.083]
 [0.109 0.02  0.    0.395 0.066 0.178 0.129 0.104]
 [0.061 0.041 0.107 0.    0.224 0.337 0.093 0.136]
 [0.076 0.07  0.031 0.124 0.    0.091 0.236 0.371]
 [0.024 0.076 0.139 0.453 0.25  0.    0.016 0.043]
 [0.01  0.024 0.049 0.738 0.037 0.04  0.    0.101]
 [0.    0.67  0.043 0.112 0.014 0.133 0.029 0.   ]]
timestep 6416 [[0.001 0.    0.071 0.761 0.016 0.06  0.017 0.073]
 [0.09  0.02  0.    0.413 0.066 0.188 0.119 0.104]
 [0.02  0.033 0.086 0.    0.491 0.296 0.044 0.03 ]
 [0.096 0.069 0.05  0.44  0.    0.218 0.046 0.081]
 [0.084 0.06  0.07  0.316 0.023 0.    

 [0.05  0.073 0.153 0.241 0.036 0.234 0.213 0.   ]]
timestep 6583 [[0.009 0.    0.114 0.419 0.    0.19  0.013 0.256]
 [0.314 0.011 0.    0.168 0.    0.065 0.084 0.357]
 [0.061 0.08  0.34  0.    0.114 0.111 0.07  0.223]
 [0.593 0.053 0.046 0.085 0.    0.14  0.06  0.024]
 [0.393 0.053 0.057 0.139 0.033 0.    0.016 0.31 ]
 [0.084 0.087 0.387 0.15  0.033 0.062 0.    0.196]
 [0.109 0.084 0.107 0.447 0.046 0.123 0.084 0.   ]]
timestep 6584 [[0.013 0.    0.067 0.448 0.    0.188 0.    0.284]
 [0.24  0.    0.    0.164 0.001 0.038 0.053 0.504]
 [0.046 0.06  0.61  0.    0.077 0.096 0.031 0.08 ]
 [0.583 0.043 0.03  0.123 0.    0.134 0.066 0.021]
 [0.044 0.043 0.057 0.55  0.04  0.    0.026 0.24 ]
 [0.02  0.063 0.046 0.681 0.056 0.029 0.    0.106]
 [0.041 0.051 0.117 0.233 0.079 0.172 0.307 0.   ]]
timestep 6585 [[0.013 0.    0.09  0.411 0.    0.184 0.    0.301]
 [0.2   0.    0.    0.173 0.009 0.033 0.057 0.529]
 [0.046 0.067 0.426 0.    0.071 0.09  0.049 0.251]
 [0.583 0.054 0.041 0.095 0.    0.141

timestep 6746 [[0.014 0.    0.104 0.434 0.    0.158 0.    0.29 ]
 [0.169 0.006 0.    0.182 0.014 0.036 0.043 0.551]
 [0.11  0.027 0.083 0.    0.074 0.44  0.086 0.18 ]
 [0.514 0.046 0.029 0.041 0.    0.064 0.251 0.054]
 [0.017 0.02  0.02  0.48  0.027 0.    0.207 0.229]
 [0.01  0.037 0.047 0.729 0.036 0.026 0.    0.114]
 [0.04  0.063 0.111 0.354 0.104 0.212 0.116 0.   ]]
timestep 6747 [[0.003 0.    0.103 0.375 0.    0.168 0.013 0.339]
 [0.167 0.    0.    0.18  0.014 0.038 0.057 0.543]
 [0.037 0.059 0.209 0.    0.059 0.07  0.059 0.509]
 [0.266 0.076 0.026 0.059 0.    0.07  0.356 0.149]
 [0.054 0.08  0.086 0.194 0.036 0.    0.183 0.367]
 [0.093 0.06  0.344 0.229 0.04  0.092 0.    0.141]
 [0.28  0.074 0.093 0.256 0.031 0.206 0.06  0.   ]]
timestep 6748 [[0.    0.    0.074 0.32  0.    0.116 0.01  0.48 ]
 [0.16  0.    0.    0.17  0.013 0.041 0.053 0.563]
 [0.043 0.537 0.147 0.    0.017 0.073 0.01  0.173]
 [0.033 0.084 0.013 0.069 0.    0.073 0.136 0.593]
 [0.059 0.08  0.13  0.056 0.023 0.    

 [0.001 0.671 0.044 0.104 0.001 0.136 0.041 0.   ]]
timestep 7051 [[0.227 0.    0.114 0.245 0.    0.138 0.061 0.214]
 [0.317 0.239 0.    0.137 0.014 0.023 0.067 0.203]
 [0.064 0.071 0.134 0.    0.267 0.26  0.109 0.094]
 [0.144 0.089 0.076 0.252 0.    0.195 0.13  0.114]
 [0.051 0.074 0.114 0.27  0.046 0.    0.023 0.421]
 [0.091 0.037 0.316 0.171 0.024 0.08  0.    0.28 ]
 [0.    0.671 0.046 0.113 0.007 0.127 0.036 0.   ]]
timestep 7052 [[0.019 0.    0.107 0.239 0.006 0.119 0.054 0.456]
 [0.176 0.107 0.    0.262 0.027 0.236 0.056 0.136]
 [0.07  0.1   0.137 0.    0.259 0.201 0.141 0.091]
 [0.124 0.086 0.074 0.316 0.    0.219 0.073 0.107]
 [0.016 0.047 0.611 0.136 0.023 0.    0.023 0.144]
 [0.08  0.04  0.399 0.178 0.04  0.064 0.    0.199]
 [0.    0.671 0.043 0.116 0.009 0.127 0.034 0.   ]]
timestep 7053 [[0.003 0.    0.099 0.565 0.016 0.131 0.059 0.129]
 [0.011 0.    0.    0.501 0.089 0.293 0.031 0.074]
 [0.066 0.087 0.13  0.    0.307 0.203 0.13  0.077]
 [0.103 0.081 0.067 0.334 0.    0.217

 [0.019 0.081 0.093 0.571 0.063 0.145 0.029 0.   ]]
timestep 7200 [[0.014 0.    0.096 0.55  0.027 0.112 0.05  0.15 ]
 [0.053 0.011 0.    0.534 0.046 0.262 0.051 0.043]
 [0.044 0.029 0.04  0.    0.086 0.057 0.057 0.687]
 [0.511 0.036 0.043 0.188 0.    0.161 0.047 0.014]
 [0.524 0.029 0.029 0.333 0.029 0.    0.014 0.043]
 [0.357 0.053 0.08  0.26  0.027 0.136 0.    0.087]
 [0.    0.043 0.063 0.724 0.049 0.093 0.029 0.   ]]
timestep 7201 [[0.031 0.    0.106 0.304 0.029 0.126 0.047 0.357]
 [0.    0.001 0.    0.642 0.036 0.247 0.037 0.037]
 [0.034 0.029 0.043 0.    0.094 0.057 0.057 0.686]
 [0.557 0.034 0.043 0.153 0.    0.148 0.05  0.014]
 [0.529 0.029 0.029 0.329 0.029 0.    0.014 0.043]
 [0.193 0.073 0.093 0.347 0.029 0.158 0.    0.109]
 [0.    0.043 0.063 0.725 0.051 0.093 0.024 0.   ]]
timestep 7202 [[0.029 0.    0.139 0.425 0.029 0.154 0.056 0.17 ]
 [0.    0.    0.    0.648 0.021 0.169 0.034 0.127]
 [0.023 0.029 0.044 0.    0.1   0.057 0.06  0.687]
 [0.263 0.041 0.033 0.313 0.    0.245